In [9]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer


In [10]:
train_file = 'blp25_hatespeech_subtask_1B_train.tsv'
validation_file = 'blp25_hatespeech_subtask_1B_dev.tsv'
test_file = 'blp25_hatespeech_subtask_1B_test.tsv'


In [11]:
#tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
# Load train/val/test DataFrames
train_df = pd.read_csv(train_file, sep="\t")
dev_df = pd.read_csv(validation_file , sep="\t")
test_df = pd.read_csv(test_file, sep="\t")





In [12]:
l2id = {'None': 0, 'Society': 1, 'Organization': 2, 'Community': 3, 'Individual': 4}
id2l = {v: k for k, v in l2id.items()}


def clean_label(x):
    # handle missing or NaN → "None"
    if pd.isna(x) or x == 'None':
        return 'None'
    # already list-like e.g. ['Abusive']
    if isinstance(x, list):
        return x[0] if len(x) > 0 else 'None'
    # string cases like "[]" or "[Abusive]" or "[Political Hate]"
    x = x.strip("[]").strip()
    if x == "":
        return 'None'
    return x


def process_df(df):
    # Ensure labels are proper lists
    df["label"] = df["label"].apply(clean_label)
    df["label"] = df["label"].fillna("None")
    # Now create binary label
    df["toxic"] = df["label"].apply(lambda x: 0 if x == "None" else 1)
    df["label_id"] = df["label"].map(l2id)

    

  

    return df

train_df = process_df(train_df)
dev_df  = process_df(dev_df)

train_df

,id,text,label,toxic,label_id
0,147963,ধন্যবাদ বর্ডার গার্ড দেরকে এভাবে পাহারা দিতে হ...,None,0,0
1,214275,ছোটবেলায় অনেক কষ্ট করে কিছু গালাগালি শিখছিলাম...,None,0,0
2,849172,অতিরিক্ত এ নিজেকে বাদুর বানাইয়া ফেলছেন রে,Individual,1,4
3,821985,চিন ভারত রাশিয়া এই তিন দেশ এক থাকলে বিশ্বকে শা...,None,0,0
4,477288,এটার বিচার কে করবেযে বিচার করবে সেই তো হলো এই ...,Individual,1,4
...,...,...,...,...,...
35517,790325,তইওয়ানের এত ক্ষমতা হয়নি যে এক টুকরো জায়গা নষ্ট...,None,0,0
35518,328377,চুরের ঘরের চুর হালা,None,0,0
35519,69803,জাহাঙ্গীর বুদ্ধি নেই মাঠে মারা যাবে,Individual,1,4
35520,419984,একটা ফেইল্ড এস্টেট এও সুষ্ঠু নির্বাচন হয় নেতার...,Organization,1,2


In [13]:

# Display the result
print(train_df[['text', 'toxic']].head())

                                                text  toxic
0  ধন্যবাদ বর্ডার গার্ড দেরকে এভাবে পাহারা দিতে হ...      0
1  ছোটবেলায় অনেক কষ্ট করে কিছু গালাগালি শিখছিলাম...      0
2          অতিরিক্ত এ নিজেকে বাদুর বানাইয়া ফেলছেন রে      1
3  চিন ভারত রাশিয়া এই তিন দেশ এক থাকলে বিশ্বকে শা...      0
4  এটার বিচার কে করবেযে বিচার করবে সেই তো হলো এই ...      1


In [14]:
train_df.columns

Index(['id', 'text', 'label', 'toxic', 'label_id'], dtype='object')

In [15]:
# Filter for toxic comments only
toxic_train_df = train_df[train_df['toxic'] == 1].copy()
toxic_dev_df = dev_df[dev_df['toxic'] == 1].copy()


# Display the result
print(toxic_train_df[['text', 'label']].head())

                                                 text       label
2           অতিরিক্ত এ নিজেকে বাদুর বানাইয়া ফেলছেন রে  Individual
4   এটার বিচার কে করবেযে বিচার করবে সেই তো হলো এই ...  Individual
7   ইরান পারমাণবিক বোমা বানাবে বানাবে বলতে বলতে বি...     Society
11  মুসলিম বাচ্চাগুলো বাচ্চা পেরে পেরে গোটা পৃথিবী...   Community
15                                   বাড়ি বাল ফালায়ছে  Individual


In [16]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Ensure your labels are numpy arrays
labels = train_df['toxic'].to_numpy()
class_names = np.unique(labels)

# Calculate weights
weights = compute_class_weight(class_weight='balanced', classes=class_names, y=labels)

print(f"Class names: {class_names}")
print(f"Calculated weights: {weights}")



Class names: [0 1]
Calculated weights: [0.83817839 1.23925481]


In [9]:
# # import torch
# # import torch.nn as nn
# # from transformers import AutoModel, AutoConfig, PreTrainedModel

# # class AttentionPooling(nn.Module):
# #     def __init__(self, hidden_size):
# #         super().__init__()
# #         self.attention = nn.Linear(hidden_size, 1)

# #     def forward(self, hidden_states, mask):
# #         # hidden_states: [batch_size, seq_len, hidden_size]
# #         # mask: [batch_size, seq_len]
        
# #         # Compute raw attention scores
# #         scores = self.attention(hidden_states).squeeze(-1)  # [batch_size, seq_len]
        
# #         # Apply mask (very important!)
# #         scores = scores.masked_fill(mask == 0, -1e9)
        
# #         # Normalize into probabilities
# #         attn_weights = torch.softmax(scores, dim=-1)  # [batch_size, seq_len]
        
# #         # Weighted sum
# #         pooled = torch.sum(hidden_states * attn_weights.unsqueeze(-1), dim=1)  # [batch_size, hidden_size]
# #         return pooled


# # class BanglaBERTWithAttention(PreTrainedModel):
# #     def __init__(self, model_name, num_labels):
# #         config = AutoConfig.from_pretrained(model_name, num_labels=num_labels)
# #         super().__init__(config)
        
# #         self.bert = AutoModel.from_pretrained(model_name, config=config)
# #         self.attention = AttentionPooling(config.hidden_size)
# #         self.dropout = nn.Dropout(config.hidden_dropout_prob)
# #         self.classifier = nn.Linear(config.hidden_size, num_labels)

# #     def forward(self, input_ids, attention_mask=None, labels=None):
# #         outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
        
# #         # Get last hidden state
# #         hidden_states = outputs.last_hidden_state  # [batch, seq_len, hidden_size]
        
# #         # Apply attention pooling
# #         pooled_output = self.attention(hidden_states, attention_mask)
# #         pooled_output = self.dropout(pooled_output)
        
# #         # Classification head
# #         logits = self.classifier(pooled_output)
        
# #         loss = None
# #         if labels is not None:
# #             loss_fn = nn.CrossEntropyLoss()
# #             loss = loss_fn(logits, labels)
        
# #         return {"loss": loss, "logits": logits}

# import torch
# import torch.nn as nn
# from transformers import AutoModel, AutoConfig, PreTrainedModel
# from transformers.modeling_outputs import SequenceClassifierOutput

# class BanglaBERTWithLSTM(PreTrainedModel):
#     def __init__(self, model_name, num_labels, class_weights=None):
#         config = AutoConfig.from_pretrained(model_name, num_labels=num_labels)
#         super().__init__(config)

#         self.bert = AutoModel.from_pretrained(model_name, config=config)

#         self.lstm = nn.LSTM(
#             input_size=config.hidden_size,
#             hidden_size=config.hidden_size,
#             num_layers=1,
#             batch_first=True,
#             bidirectional=False
#         )

#         classifier_input_size = config.hidden_size * 2  # CLS + LSTM hidden

#         self.classifier_head = nn.Sequential(
#             nn.LayerNorm(classifier_input_size),
#             nn.Linear(classifier_input_size, config.hidden_size),
#             nn.GELU(),
#             nn.Dropout(config.hidden_dropout_prob),
#             nn.Linear(config.hidden_size, num_labels)
#         )

#         # ✅ Properly register class weights
#         if class_weights is not None:
#             self.register_buffer("class_weights", torch.tensor(class_weights, dtype=torch.float))
#         else:
#             self.class_weights = None

#     def forward(self, input_ids, attention_mask=None, labels=None):
#         outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
#         hidden_states = outputs.last_hidden_state  # [batch, seq, hidden]

#         cls_output = hidden_states[:, 0, :]  # [batch, hidden]

#         # ✅ Use final hidden state from LSTM instead of last timestep
#         _, (h_n, _) = self.lstm(hidden_states)
#         lstm_pooled = h_n[-1]  # [batch, hidden]

#         combined_output = torch.cat((cls_output, lstm_pooled), dim=1)

#         logits = self.classifier_head(combined_output)

#         loss = None
#         if labels is not None:
#             labels = labels.long()
#             loss_fn = nn.CrossEntropyLoss(weight=self.class_weights)
#             loss = loss_fn(logits, labels)

#         return SequenceClassifierOutput(
#             loss=loss,
#             logits=logits,
#             hidden_states=outputs.hidden_states,
#             attentions=outputs.attentions,
#         )


In [18]:
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    Trainer, 
    TrainingArguments,
    DataCollatorWithPadding  # <-- Import the data collator
)
from datasets import Dataset
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

# --- 1. Load Tokenizer and Model ---
model_name = "csebuetnlp/banglabert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_1 = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


# def truncate_long_texts(df, max_chars=10000):
#     df['text'] = df['text'].astype(str).apply(lambda x: x[:max_chars])
#     return df

# train_df = truncate_long_texts(train_df)
# dev_df = truncate_long_texts(dev_df)

# --- 2. Create Datasets ---
train_dataset_1 = Dataset.from_pandas(train_df[['text', 'toxic']].rename(columns={'toxic': 'label'}))
dev_dataset_1 = Dataset.from_pandas(dev_df[['text', 'toxic']].rename(columns={'toxic': 'label'}))

def tokenize(batch):
    return tokenizer(batch['text'], truncation=True, max_length=512)



train_dataset_1 = train_dataset_1.map(tokenize, batched=True)
dev_dataset_1 = dev_dataset_1.map(tokenize, batched=True)

# Set the format to torch tensors and specify columns
train_dataset_1.set_format("torch", columns=["input_ids", "attention_mask", "label"])
dev_dataset_1.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# --- 3. Define Training ---

# Create a data collator that will dynamically pad the batches
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def compute_metrics_binary(p):
    preds = np.argmax(p.predictions, axis=1)
    return {"accuracy": accuracy_score(p.label_ids, preds), "f1": f1_score(p.label_ids, preds)}

training_args_1 = TrainingArguments(
    output_dir="./results_model_1",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    
    logging_dir="./logs_1",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer_1 = Trainer(
    model=model_1,
    args=training_args_1,
    train_dataset=train_dataset_1,
    eval_dataset=dev_dataset_1,
    compute_metrics=compute_metrics_binary,
    data_collator=data_collator, # <-- Add the data collator here
)

# --- 4. Start Fine-tuning ---
trainer_1.train()
trainer_1.save_model("./final_model_task2_1")

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at csebuetnlp/banglabert and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/35522 [00:00<?, ? examples/s]

Map:   0%|          | 0/2512 [00:00<?, ? examples/s]

ValueError: --load_best_model_at_end requires the save and eval strategy to match, but found
- Evaluation strategy: IntervalStrategy.NO
- Save strategy: SaveStrategy.EPOCH

In [11]:
# from transformers import (
#     AutoTokenizer,
#     AutoModelForSequenceClassification,
#     Trainer,
#     TrainingArguments,
#     DataCollatorWithPadding
# )
# from datasets import Dataset
# import numpy as np
# from sklearn.metrics import accuracy_score, f1_score

# # --- 1) Model & tokenizer ---
# model_name = "FacebookAI/xlm-roberta-base"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model_1 = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# # --- 2) Datasets (ensure integer labels) ---
# train_df = train_df.copy()
# dev_df   = dev_df.copy()
# train_df["toxic"] = train_df["toxic"].astype(int)
# dev_df["toxic"]   = dev_df["toxic"].astype(int)

# train_dataset_1 = Dataset.from_pandas(train_df[["text", "toxic"]].rename(columns={"toxic": "label"}))
# dev_dataset_1   = Dataset.from_pandas(dev_df[["text", "toxic"]].rename(columns={"toxic": "label"}))

# def tokenize(batch):
#     return tokenizer(batch["text"], truncation=True, max_length=512)

# train_dataset_1 = train_dataset_1.map(tokenize, batched=True)
# dev_dataset_1   = dev_dataset_1.map(tokenize, batched=True)

# train_dataset_1.set_format("torch", columns=["input_ids", "attention_mask", "label"])
# dev_dataset_1.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# # --- 3) Training setup ---
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# def compute_metrics_binary(p):
#     preds = np.argmax(p.predictions, axis=1)
#     return {"accuracy": accuracy_score(p.label_ids, preds),
#             "f1": f1_score(p.label_ids, preds)}

# training_args_1 = TrainingArguments(
#     output_dir="./results_model_1",
#     num_train_epochs=5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     eval_strategy="epoch",                 # <-- renamed arg
#     save_strategy="epoch",
#     logging_dir="./logs_1",
#     load_best_model_at_end=True,
#     metric_for_best_model="f1",
#     greater_is_better=True,
#     save_total_limit=2,
#     report_to="none",                      # or "tensorboard" if you use it
#     disable_tqdm=False                     # set True if widgets annoy; or install ipywidgets
# )

# trainer_1 = Trainer(
#     model=model_1,
#     args=training_args_1,
#     train_dataset=train_dataset_1,
#     eval_dataset=dev_dataset_1,
#     compute_metrics=compute_metrics_binary,
#     data_collator=data_collator,
# )

# trainer_1.train()
# trainer_1.save_model("./final_model_1")


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/35522 [00:00<?, ? examples/s]

Map:   0%|          | 0/2512 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.682100,0.668649,0.611465,0.000000
2,0.675900,0.668298,0.611465,0.000000
3,0.677400,0.668471,0.611465,0.000000


KeyboardInterrupt: 

In [11]:
toxic_train_df

,id,text,label,toxic,label_id
2,849172,অতিরিক্ত এ নিজেকে বাদুর বানাইয়া ফেলছেন রে,Individual,1,4
4,477288,এটার বিচার কে করবেযে বিচার করবে সেই তো হলো এই ...,Individual,1,4
7,786609,ইরান পারমাণবিক বোমা বানাবে বানাবে বলতে বলতে বি...,Society,1,1
11,124917,মুসলিম বাচ্চাগুলো বাচ্চা পেরে পেরে গোটা পৃথিবী...,Community,1,3
15,432369,বাড়ি বাল ফালায়ছে,Individual,1,4
...,...,...,...,...,...
35512,343784,এই বাজেট গরিব মারার বাজেট,Organization,1,2
35514,668158,কৃষি মন্ত্রীর সারের ব্যবসা করে তাই এ অবস্থা,Individual,1,4
35519,69803,জাহাঙ্গীর বুদ্ধি নেই মাঠে মারা যাবে,Individual,1,4
35520,419984,একটা ফেইল্ড এস্টেট এও সুষ্ঠু নির্বাচন হয় নেতার...,Organization,1,2


In [12]:
toxic_dev_df

,id,text,label,toxic,label_id
0,166449,ইন্ডিয়া কি মাছ ধরা বন্ধ রাখছেএক নদীতে দুইনীতি ...,Society,1,1
1,267692,লক্ষ টাকা ঘুষ দিয়ে অযোগ্য আর দায়িত্বহীন মানস...,Organization,1,2
3,939131,আর কতো শিখবে আমার সোনার ছেলেরা এগুলো কে টাকা দ...,Individual,1,4
4,210284,কি সাংঘাতিক ভাই রে তুই,Individual,1,4
5,712332,লঞ্চ মালিকদের অভিশপ্ত চক্ষু পদ্মা সেতুর উপর,Community,1,3
...,...,...,...,...,...
2496,653048,কিরে মানিক চোরা তুইও আছোস,Individual,1,4
2503,121961,দেশের সবই তো চুরি হয়ে যাচ্ছে আর চোরদের ধরার কো...,Organization,1,2
2504,555021,ফকিনি বাংলা দেশ কারেন্ট নাই,Society,1,1
2505,858412,কাকু সামনে চশমা থাকবে কিন্তু গ্লাস থাকবেনাপাছা...,Individual,1,4


In [13]:

l2id = {

    'Society': 0, 'Organization': 1, 'Community': 2, 'Individual': 3

}
id2l = {v: k for k, v in l2id.items()}


toxic_train_df["label"] = toxic_train_df["label"].map(l2id)
toxic_train_df['label'].unique()

toxic_dev_df["label"] = toxic_dev_df["label"].map(l2id)
toxic_dev_df['label'].unique()

array([0, 1, 3, 2])

In [14]:
toxic_train_df

,id,text,label,toxic,label_id
2,849172,অতিরিক্ত এ নিজেকে বাদুর বানাইয়া ফেলছেন রে,3,1,4
4,477288,এটার বিচার কে করবেযে বিচার করবে সেই তো হলো এই ...,3,1,4
7,786609,ইরান পারমাণবিক বোমা বানাবে বানাবে বলতে বলতে বি...,0,1,1
11,124917,মুসলিম বাচ্চাগুলো বাচ্চা পেরে পেরে গোটা পৃথিবী...,2,1,3
15,432369,বাড়ি বাল ফালায়ছে,3,1,4
...,...,...,...,...,...
35512,343784,এই বাজেট গরিব মারার বাজেট,1,1,2
35514,668158,কৃষি মন্ত্রীর সারের ব্যবসা করে তাই এ অবস্থা,3,1,4
35519,69803,জাহাঙ্গীর বুদ্ধি নেই মাঠে মারা যাবে,3,1,4
35520,419984,একটা ফেইল্ড এস্টেট এও সুষ্ঠু নির্বাচন হয় নেতার...,1,1,2


In [15]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Ensure your labels are numpy arrays
labels = toxic_train_df['label'].to_numpy()
class_names = np.unique(labels)

# Calculate weights
weights = compute_class_weight(class_weight='balanced', classes=class_names, y=labels)

print(f"Class names: {class_names}")
print(f"Calculated weights: {weights}")



Class names: [0 1 2 3]
Calculated weights: [1.62494331 0.93161726 1.3597723  0.63460857]


In [31]:
# # # import torch
# # # import torch.nn as nn
# # # from transformers import AutoModel, AutoConfig, PreTrainedModel

# # # class AttentionPooling(nn.Module):
# # #     def __init__(self, hidden_size):
# # #         super().__init__()
# # #         self.attention = nn.Linear(hidden_size, 1)

# # #     def forward(self, hidden_states, mask):
# # #         # hidden_states: [batch_size, seq_len, hidden_size]
# # #         # mask: [batch_size, seq_len]
        
# # #         # Compute raw attention scores
# # #         scores = self.attention(hidden_states).squeeze(-1)  # [batch_size, seq_len]
        
# # #         # Apply mask (very important!)
# # #         scores = scores.masked_fill(mask == 0, -1e9)
        
# # #         # Normalize into probabilities
# # #         attn_weights = torch.softmax(scores, dim=-1)  # [batch_size, seq_len]
        
# # #         # Weighted sum
# # #         pooled = torch.sum(hidden_states * attn_weights.unsqueeze(-1), dim=1)  # [batch_size, hidden_size]
# # #         return pooled


# # # class BanglaBERTWithAttention(PreTrainedModel):
# # #     def __init__(self, model_name, num_labels):
# # #         config = AutoConfig.from_pretrained(model_name, num_labels=num_labels)
# # #         super().__init__(config)
        
# # #         self.bert = AutoModel.from_pretrained(model_name, config=config)
# # #         self.attention = AttentionPooling(config.hidden_size)
# # #         self.dropout = nn.Dropout(config.hidden_dropout_prob)
# # #         self.classifier = nn.Linear(config.hidden_size, num_labels)

# # #     def forward(self, input_ids, attention_mask=None, labels=None):
# # #         outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
        
# # #         # Get last hidden state
# # #         hidden_states = outputs.last_hidden_state  # [batch, seq_len, hidden_size]
        
# # #         # Apply attention pooling
# # #         pooled_output = self.attention(hidden_states, attention_mask)
# # #         pooled_output = self.dropout(pooled_output)
        
# # #         # Classification head
# # #         logits = self.classifier(pooled_output)
        
# # #         loss = None
# # #         if labels is not None:
# # #             loss_fn = nn.CrossEntropyLoss()
# # #             loss = loss_fn(logits, labels)
        
# # #         return {"loss": loss, "logits": logits}


# # import torch
# # import torch.nn as nn
# # from transformers import AutoModel, AutoConfig, PreTrainedModel
# # from transformers.modeling_outputs import SequenceClassifierOutput


# # class MultiHeadAttentionPooling(nn.Module):
# #     def __init__(self, hidden_size, num_heads=4):
# #         super().__init__()
# #         self.num_heads = num_heads
# #         self.proj = nn.Linear(hidden_size, num_heads)
# #         nn.init.xavier_uniform_(self.proj.weight)
# #         if self.proj.bias is not None:
# #             nn.init.zeros_(self.proj.bias)

# #     def forward(self, hidden_states, mask):
# #         # hidden_states: [batch, seq, hidden]
# #         # mask: [batch, seq]
# #         scores = self.proj(hidden_states)  # [batch, seq, heads]
# #         scores = scores.masked_fill(mask.unsqueeze(-1) == 0, -1e9)

# #         attn_weights = torch.softmax(scores, dim=1)  # [batch, seq, heads]

# #         # Weighted sum per head
# #         pooled = torch.einsum(
# #             "bsh,bsd->bhd", attn_weights, hidden_states
# #         )  # [batch, heads, hidden]

# #         return pooled.reshape(hidden_states.size(0), -1)  # concat heads


# # class BanglaBERTWithAttention(PreTrainedModel):
# #     def __init__(self, model_name, num_labels, class_weights=None, num_heads=4):
# #         config = AutoConfig.from_pretrained(model_name, num_labels=num_labels)
# #         super().__init__(config)

# #         self.bert = AutoModel.from_pretrained(model_name, config=config)
# #         self.attention = MultiHeadAttentionPooling(config.hidden_size, num_heads=num_heads)
# #         self.norm = nn.LayerNorm(config.hidden_size * num_heads)
# #         self.dropout = nn.Dropout(config.hidden_dropout_prob)
# #         self.classifier = nn.Linear(config.hidden_size * num_heads, num_labels)

# #         # Handle class imbalance
# #         if class_weights is not None:
# #             self.register_buffer("class_weights", torch.tensor(class_weights))
# #         else:
# #             self.class_weights = None

# #     def forward(self, input_ids, attention_mask=None, labels=None):
# #         outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)

# #         hidden_states = outputs.last_hidden_state  # [batch, seq, hidden]

# #         pooled_output = self.attention(hidden_states, attention_mask)
# #         pooled_output = self.norm(pooled_output)
# #         pooled_output = self.dropout(pooled_output)

# #         logits = self.classifier(pooled_output)

# #         if labels is not None:
# #             if self.config.num_labels == 1:
# #                 # regression
# #                 loss_fn = nn.MSELoss()
# #                 loss = loss_fn(logits.view(-1), labels.view(-1))
# #             else:
# #                 # classification
# #                 labels = labels.long()   # <---- important fix
# #                 loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1)
# #                 loss = loss_fn(logits, labels)


# #         return SequenceClassifierOutput(
# #             loss=loss,
# #             logits=logits,
# #             hidden_states=outputs.hidden_states,
# #             attentions=outputs.attentions,
# #         )




# # import torch
# # import torch.nn as nn
# # from transformers import AutoModel, AutoConfig, PreTrainedModel

# # class AttentionPooling(nn.Module):
# #     def __init__(self, hidden_size):
# #         super().__init__()
# #         self.attention = nn.Linear(hidden_size, 1)

# #     def forward(self, hidden_states, mask):
# #         # hidden_states: [batch_size, seq_len, hidden_size]
# #         # mask: [batch_size, seq_len]
        
# #         # Compute raw attention scores
# #         scores = self.attention(hidden_states).squeeze(-1)  # [batch_size, seq_len]
        
# #         # Apply mask (very important!)
# #         scores = scores.masked_fill(mask == 0, -1e9)
        
# #         # Normalize into probabilities
# #         attn_weights = torch.softmax(scores, dim=-1)  # [batch_size, seq_len]
        
# #         # Weighted sum
# #         pooled = torch.sum(hidden_states * attn_weights.unsqueeze(-1), dim=1)  # [batch_size, hidden_size]
# #         return pooled


# # class BanglaBERTWithAttention(PreTrainedModel):
# #     def __init__(self, model_name, num_labels):
# #         config = AutoConfig.from_pretrained(model_name, num_labels=num_labels)
# #         super().__init__(config)
        
# #         self.bert = AutoModel.from_pretrained(model_name, config=config)
# #         self.attention = AttentionPooling(config.hidden_size)
# #         self.dropout = nn.Dropout(config.hidden_dropout_prob)
# #         self.classifier = nn.Linear(config.hidden_size, num_labels)

# #     def forward(self, input_ids, attention_mask=None, labels=None):
# #         outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
        
# #         # Get last hidden state
# #         hidden_states = outputs.last_hidden_state  # [batch, seq_len, hidden_size]
        
# #         # Apply attention pooling
# #         pooled_output = self.attention(hidden_states, attention_mask)
# #         pooled_output = self.dropout(pooled_output)
        
# #         # Classification head
# #         logits = self.classifier(pooled_output)
        
# #         loss = None
# #         if labels is not None:
# #             loss_fn = nn.CrossEntropyLoss()
# #             loss = loss_fn(logits, labels)
        
# #         return {"loss": loss, "logits": logits}


# import torch
# import torch.nn as nn
# from transformers import AutoModel, AutoConfig, PreTrainedModel
# from transformers.modeling_outputs import SequenceClassifierOutput


# import torch
# import torch.nn as nn
# from transformers import AutoModel, AutoConfig, PreTrainedModel
# from transformers.modeling_outputs import SequenceClassifierOutput

# # Your MultiHeadAttentionPooling class remains the same, it's well-implemented.
# class MultiHeadAttentionPooling(nn.Module):
#     def __init__(self, hidden_size, num_heads=4):
#         super().__init__()
#         self.num_heads = num_heads
#         self.proj = nn.Linear(hidden_size, num_heads)
#         nn.init.xavier_uniform_(self.proj.weight)
#         if self.proj.bias is not None:
#             nn.init.zeros_(self.proj.bias)

#     def forward(self, hidden_states, mask):
#         # hidden_states: [batch, seq, hidden]
#         # mask: [batch, seq]
#         scores = self.proj(hidden_states)  # [batch, seq, heads]
#         scores = scores.masked_fill(mask.unsqueeze(-1) == 0, -1e9)
#         attn_weights = torch.softmax(scores, dim=1)  # [batch, seq, heads]
#         # Weighted sum per head
#         pooled = torch.einsum(
#             "bsh,bsd->bhd", attn_weights, hidden_states
#         )  # [batch, heads, hidden]
#         return pooled.reshape(hidden_states.size(0), -1)  # concat heads


# class BanglaBERTWithAttention(PreTrainedModel):
#     # ... (your existing __init__ method) ...

#     def forward(self, input_ids, attention_mask=None, labels=None):
#         outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)

#         hidden_states = outputs.last_hidden_state
#         cls_output = hidden_states[:, 0, :]
#         attention_pooled_output = self.attention(hidden_states, attention_mask)
#         combined_output = torch.cat((cls_output, attention_pooled_output), dim=1)
#         logits = self.classifier_head(combined_output)

#         loss = None
#         if labels is not None:
#             # ⭐️ Fix: Ensure the labels are of type torch.long
#             labels = labels.long()
#             loss_fn = nn.CrossEntropyLoss(weight=self.class_weights)
#             loss = loss_fn(logits, labels)

#         return SequenceClassifierOutput(
#             loss=loss,
#             logits=logits,
#             hidden_states=outputs.hidden_states,
#             attentions=outputs.attentions,
#         )

#     def forward(self, input_ids, attention_mask=None, labels=None):
#         outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
        
#         hidden_states = outputs.last_hidden_state  # [batch, seq, hidden]
        
#         # ⭐️ 1. Get the standard [CLS] token representation
#         cls_output = hidden_states[:, 0, :]  # [batch, hidden]
        
#         # 2. Get your custom attention-pooled representation
#         attention_pooled_output = self.attention(hidden_states, attention_mask)
        
#         # ⭐️ 3. Concatenate both representations to combine their strengths
#         combined_output = torch.cat((cls_output, attention_pooled_output), dim=1)
        
#         # 4. Pass the combined representation through the new classifier head
#         logits = self.classifier_head(combined_output)

#         loss = None
#         if labels is not None:
#             loss_fn = nn.CrossEntropyLoss(weight=self.class_weights)
#             loss = loss_fn(logits, labels)

#         return SequenceClassifierOutput(
#             loss=loss,
#             logits=logits,
#             hidden_states=outputs.hidden_states,
#             attentions=outputs.attentions,
#         )

In [28]:
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    Trainer, 
    TrainingArguments,
    DataCollatorWithPadding  # <-- Import the data collator
)
from datasets import Dataset
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

# --- 1. Load Tokenizer and Model ---
model_name = "csebuetnlp/banglabert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_2 = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)


# def truncate_long_texts(df, max_chars=10000):
#     df['text'] = df['text'].astype(str).apply(lambda x: x[:max_chars])
#     return df

# train_df = truncate_long_texts(train_df)
# dev_df = truncate_long_texts(dev_df)

# --- 2. Create Datasets ---
train_dataset_2 = Dataset.from_pandas(toxic_train_df[['text', 'label']])
dev_dataset_2 = Dataset.from_pandas(toxic_dev_df[['text', 'label']])

def tokenize(batch):
    return tokenizer(batch['text'], truncation=True, max_length=512)



train_dataset_2 = train_dataset_2.map(tokenize, batched=True)
dev_dataset_2 = dev_dataset_2.map(tokenize, batched=True)

# Set the format to torch tensors and specify columns
train_dataset_2.set_format("torch", columns=["input_ids", "attention_mask", "label"])
dev_dataset_2.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# --- 3. Define Training ---

# Create a data collator that will dynamically pad the batches
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def compute_metrics_multiclass(p):
    preds = np.argmax(p.predictions, axis=1)
    return {
        "accuracy": accuracy_score(p.label_ids, preds),
        "f1_weighted": f1_score(p.label_ids, preds, average="weighted"),
    }

training_args_2 = TrainingArguments(
    output_dir="./results_model_2",
    num_train_epochs=15,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    logging_dir="./logs_1",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer_2 = Trainer(
    model=model_2,
    args=training_args_2,
    train_dataset=train_dataset_2,
    eval_dataset=dev_dataset_2,
    compute_metrics=compute_metrics_multiclass,
    data_collator=data_collator, # <-- Add the data collator here
)

# --- 4. Start Fine-tuning ---
trainer_2.train()
trainer_2.save_model("./final_model_task2_2")

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at csebuetnlp/banglabert and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/14332 [00:00<?, ? examples/s]

Map:   0%|          | 0/976 [00:00<?, ? examples/s]

/users/t/p/tprama/.local/lib/python3.12/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Weighted
1,0.954900,0.760744,0.707992,0.708658
2,0.690300,0.785590,0.716189,0.716537
3,0.534000,0.912051,0.716189,0.715709
4,0.379400,1.061915,0.702869,0.706373
5,0.292700,1.287596,0.693648,0.695726
6,0.199300,1.385498,0.700820,0.701336
7,0.161200,1.642108,0.692623,0.691757
8,0.119900,1.753495,0.698770,0.699908
9,0.082300,1.947013,0.684426,0.688866
10,0.066200,2.165715,0.703893,0.707877


In [18]:
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    Trainer, 
    TrainingArguments,
    DataCollatorWithPadding  # <-- Import the data collator
)
from datasets import Dataset
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
# print("Tokenizer vocab size:", len(tokenizer))
# print("Model embedding size:", pipe_1.model.get_input_embeddings().weight.shape[0])
model_name = "csebuetnlp/banglabert"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [19]:
from transformers import pipeline
from sklearn.metrics import classification_report
import numpy as np

# --- 1. Load the fine-tuned models with truncation ---
pipe_1 = pipeline(
    "text-classification",
    model="./final_model_task2_1",
    tokenizer=tokenizer,
    device=-1,   # force CPU
    truncation=True,
    max_length=512
)

pipe_2 = pipeline(
    "text-classification",
    model="./final_model_task2_2",
    tokenizer=tokenizer,
    device=-1,
    truncation=True,
    max_length=512,
    return_all_scores=True
)

# --- 2. Run the pipeline on the test set ---
predictions = []
for text in test_df['text']:
    # Model 1: Is it toxic?
    result_1 = pipe_1(text, truncation=True, max_length=512)[0]
    print(result_1)
    is_toxic = 1 if result_1['label'] == 'LABEL_1' else 0
    print(is_toxic)
    
    pred_labels = [0] * pipe_2.model.config.num_labels  # default: all zero
    
    if is_toxic:
        # Model 2: Which toxic class? (highest score = predicted label)
        result_2 = pipe_2(text, truncation=True, max_length=512)[0]
        best_label = max(result_2, key=lambda x: x['score'])
        pred_labels = [1 if r['label'] == best_label['label'] else 0 for r in result_2]
    
    predictions.append(pred_labels)

y_pred = np.array(predictions)
print(y_pred.shape)


Device set to use cpu
Device set to use cpu
/users/t/p/tprama/.local/lib/python3.12/site-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


{'label': 'LABEL_0', 'score': 0.9765249490737915}
0
{'label': 'LABEL_1', 'score': 0.9295563101768494}
1
{'label': 'LABEL_0', 'score': 0.8381099104881287}
0
{'label': 'LABEL_0', 'score': 0.9434114694595337}
0
{'label': 'LABEL_0', 'score': 0.535946249961853}
0
{'label': 'LABEL_1', 'score': 0.5558633804321289}
1
{'label': 'LABEL_0', 'score': 0.9515287280082703}
0
{'label': 'LABEL_1', 'score': 0.6490920782089233}
1
{'label': 'LABEL_1', 'score': 0.5236160755157471}
1
{'label': 'LABEL_1', 'score': 0.5666013360023499}
1
{'label': 'LABEL_0', 'score': 0.8919270634651184}
0
{'label': 'LABEL_0', 'score': 0.949476420879364}
0
{'label': 'LABEL_0', 'score': 0.5738127827644348}
0
{'label': 'LABEL_1', 'score': 0.5299589037895203}
1
{'label': 'LABEL_1', 'score': 0.7250430583953857}
1
{'label': 'LABEL_1', 'score': 0.5877763628959656}
1
{'label': 'LABEL_0', 'score': 0.7817710041999817}
0
{'label': 'LABEL_0', 'score': 0.8197878003120422}
0
{'label': 'LABEL_0', 'score': 0.9567469358444214}
0
{'label': 'LAB

In [62]:
# # from transformers import pipeline
# # from sklearn.metrics import classification_report
# # import numpy as np

# # pipe_1 = pipeline(
# #     "text-classification",
# #     model="./final_model_1",
# #     tokenizer=tokenizer,
# #     truncation=True,
# #     max_length=512,
# #     device=-1,
# # )

# # pipe_2 = pipeline(
# #     "text-classification",
# #     model="./final_model_2",
# #     tokenizer=tokenizer,
# #     device=-1,
# #     truncation=True,
# #     max_length=512,
# #     return_all_scores=True
# # )

# # --- 2. Run the pipeline on the test set ---
# predictions = []
# for text in test_df['text']:
#     # # Model 1: Is it toxic?
#     # print(text)
#     # result_1 = pipe_1(text)[0]
#     is_toxic = 1 
#     #if result_1['label'] == 'LABEL_1' else 0
    
#     pred_labels = [0] * pipe_2.model.config.num_labels  # default: all zero
    
#     if is_toxic:
#         # Model 2: Which toxic class? (highest score = predicted label)
#         result_2 = pipe_2(text, truncation=True, max_length=512)[0]
#         best_label = max(result_2, key=lambda x: x['score'])
#         pred_labels = [1 if r['label'] == best_label['label'] else 0 for r in result_2]
    
#     predictions.append(pred_labels)

# y_pred = np.array(predictions)
# print(y_pred.shape)


In [20]:
test_df

,id,text
0,879187,শুভ কামনা রইল বাংলাদেশ জন্য ইনশাআল্লাহ জয় হবে
1,316919,গোয়া মারা দিয়ে আছে বাংলাদেশ মাদারচোদ নিউজ করে ...
2,916242,ভাইয়া আপনি অভিনেতা হইয়েন না না হলে সবাই বাচ্...
3,786824,আমাদেরো তাই দেখছি
4,47284,পুলিশ কতটা টাকা নিয়ে
...,...,...
2507,776466,সত্য কথা তেতু লাগে
2508,849227,এই ফকিননি মাগীটা আর কত নাটক দেখাবে
2509,532697,দেখো আজকে কার ফিটনেস কোথায় দাঁড়িয়েছে তুমি চ...
2510,861411,ছোট ভাইটির পাসে থাকুন গেম ভিড়িও বানাই


In [21]:
import numpy as np
import pandas as pd

# Your mapping
id2l = {
   0: 'Society', 1: 'Organization', 2: 'Community', 3:'Individual'
}

# Example y_pred
# y_pred = np.array([[0,0,0,0,0],[0,0,0,1,0],[1,0,0,0,0]])

def decode_labels(row):
    indices = np.where(row == 1)[0]
    if len(indices) == 0:
        return "None"
    # If multiple labels, join them with comma
    return ", ".join([id2l[i] for i in indices])

# Convert predictions into a DataFrame column
df = pd.DataFrame()
df["Predicted_Label"] = [decode_labels(row) for row in y_pred]

print(df.head())


  Predicted_Label
0            None
1      Individual
2            None
3            None
4            None


In [22]:
df

,Predicted_Label
0,None
1,Individual
2,None
3,None
4,None
...,...
2507,None
2508,Individual
2509,Individual
2510,None


In [23]:
test_df['label']=df['Predicted_Label']
test_df['model']='bangla-bert'
test_df

,id,text,label,model
0,879187,শুভ কামনা রইল বাংলাদেশ জন্য ইনশাআল্লাহ জয় হবে,None,bangla-bert
1,316919,গোয়া মারা দিয়ে আছে বাংলাদেশ মাদারচোদ নিউজ করে ...,Individual,bangla-bert
2,916242,ভাইয়া আপনি অভিনেতা হইয়েন না না হলে সবাই বাচ্...,None,bangla-bert
3,786824,আমাদেরো তাই দেখছি,None,bangla-bert
4,47284,পুলিশ কতটা টাকা নিয়ে,None,bangla-bert
...,...,...,...,...
2507,776466,সত্য কথা তেতু লাগে,None,bangla-bert
2508,849227,এই ফকিননি মাগীটা আর কত নাটক দেখাবে,Individual,bangla-bert
2509,532697,দেখো আজকে কার ফিটনেস কোথায় দাঁড়িয়েছে তুমি চ...,Individual,bangla-bert
2510,861411,ছোট ভাইটির পাসে থাকুন গেম ভিড়িও বানাই,None,bangla-bert


In [24]:
#test_df['model']='bert-base-multilingual-cased'

In [25]:
test_df.columns

Index(['id', 'text', 'label', 'model'], dtype='object')

In [26]:
test_df = test_df[['id', 'label', 'model']]
test_df

,id,label,model
0,879187,None,bangla-bert
1,316919,Individual,bangla-bert
2,916242,None,bangla-bert
3,786824,None,bangla-bert
4,47284,None,bangla-bert
...,...,...,...
2507,776466,None,bangla-bert
2508,849227,Individual,bangla-bert
2509,532697,Individual,bangla-bert
2510,861411,None,bangla-bert


In [27]:
test_df.to_csv("subtask_2A.tsv", sep="\t", index=False)
print("Saved to final_ensemble.tsv")

Saved to final_ensemble.tsv
